# Preparation

In [20]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../../../../') # The simulator package and package needed for analysis 
                                  # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

In [ ]:
vehicle_rate = 22

## Simulation setup

### Simulation params setup

In [21]:
# Specify the labels of the road segments in each route
all_routes = [[0, 1], [2, 3]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.5, 0.5]]*vehicle_limit))

### Topology setup

In [23]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

### Utility params setup

In [24]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 0

## Function needed for running simulation

In [25]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go through these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [26]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 1min 59s, sys: 2.34 s, total: 2min 2s
Wall time: 2min 5s


In [27]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]"
1,1,"[0, 1]",70.65,"[0, 0]","[0, 0]","[0, 0]"
2,2,"[0, 1]",73.0,"[1, 1]","[0, 0]","[0, 0]"
3,3,"[2, 3]",67.8,"[-999, -999]","[0, 0]","[0, 0]"
4,4,"[2, 3]",71.3,"[3, 3]","[0, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[0, 1]",80.3,"[73, 73]","[0, 0]","[0, 0]"
76,76,"[2, 3]",69.55,"[74, 74]","[0, 0]","[0, 0]"
77,77,"[0, 1]",79.8,"[75, 75]","[0, 0]","[0, 0]"
78,78,"[2, 3]",70.1,"[76, 76]","[0, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [28]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]",1
1,1,"[0, 1]",70.65,"[0, 0]","[0, 0]","[0, 0]",1
2,2,"[0, 1]",73.0,"[1, 1]","[0, 0]","[0, 0]",1
3,3,"[2, 3]",67.8,"[-999, -999]","[0, 0]","[0, 0]",1
4,4,"[2, 3]",71.3,"[3, 3]","[0, 0]","[0, 0]",1
5,5,"[0, 1]",69.75,"[2, 2]","[0, 0]","[0, 0]",1
6,6,"[2, 3]",71.2,"[4, 4]","[0, 0]","[0, 0]",1
7,7,"[0, 1]",69.55,"[5, 5]","[0, 0]","[0, 0]",1
8,8,"[2, 3]",70.9,"[6, 6]","[0, 0]","[0, 0]",1
9,9,"[2, 3]",73.25,"[8, 8]","[0, 0]","[0, 0]",1


### Set up dataframe

In [29]:
utility_df = pd.read_csv("../2.1.2.1 Zero alpha/Saved_data/Utility_data/Round_1.csv")

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0]","[0, 0]",[],4.387328,0.591669
1,1,"[0, 0]","[0, 0]",[],4.756538,0.426077
2,2,"[0, 0]","[0, 0]",[],4.831770,0.477468
3,3,"[0, 0]","[0, 0]",[],5.291330,0.409165
4,4,"[0, 0]","[0, 0]",[],5.045563,0.403231


## Summary of the 1st round

In [30]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,0,0.0,1,67.45
1,1,0,0.0,1,70.65
2,2,0,0.0,1,73.00
3,3,1,0.0,1,67.80
4,4,1,0.0,1,71.30
5,5,0,0.0,1,69.75
6,6,1,0.0,1,71.20
7,7,0,0.0,1,69.55
8,8,1,0.0,1,70.90
9,9,1,0.0,1,73.25


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[-0.5440394379715727, 16.651092223153956]","[3.40603564917406e-08, 0.9999999659396435]",[0],4.387328,0.591669
1,1,"[-0.015455900416226909, 16.651092223153956]","[5.778433138313833e-08, 0.9999999422156687]",[0],4.756538,0.426077
2,2,"[-0.15936493480221814, 16.651092223153956]","[5.0039296610289195e-08, 0.9999999499607034]",[0],4.831770,0.477468
3,3,"[16.651092223153956, 0.02495796323314381]","[0.9999999398325495, 6.016745050826398e-08]",[1],5.291330,0.409165
4,4,"[16.651092223153956, 0.03940595608673125]","[0.9999999389569404, 6.104305951500339e-08]",[1],5.045563,0.403231
5,5,"[-1.1382212747108034, 16.651092223153956]","[1.8801794553990453e-08, 0.9999999811982054]",[0],4.728321,0.709449
6,6,"[16.651092223153956, -0.27553215618123383]","[0.9999999554486964, 4.455130351389479e-08]",[1],6.209452,0.510895
7,7,"[-0.05076341952933827, 16.651092223153956]","[5.577970750629203e-08, 0.9999999442202926]",[0],4.302608,0.441316
8,8,"[16.651092223153956, -0.20381398876988]","[0.9999999521361952, 4.7863804766542366e-08]",[1],5.180132,0.491483
9,9,"[16.651092223153956, -0.13735745539261335]","[0.9999999488472576, 5.1152742473068006e-08]",[1],4.686470,0.470395


In [31]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [32]:
%%time

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)    
    vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)

CPU times: user 1h 42min 7s, sys: 1min 56s, total: 1h 44min 3s
Wall time: 1h 42min 47s
